In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import glob
import sys
import pandasql as pdsql

sys.path.append("..")
import analysislib as alib

In [36]:
master3Processes = pd.read_csv("../../data/2025-03-11-time-14-55-15-third-run-process-scaling/master.csv", index_col=0)
master3Limmem1 = pd.read_csv("../../data/2025-03-11-time-21-44-10-third-run-limmem/master.csv", index_col=0)

In [37]:
sq = lambda q: pdsql.sqldf(q, globals())

In [38]:
print(open("../../data/2025-03-11-time-14-55-15-third-run-process-scaling/fio-config.txt").read())

Total file size during runs: 34359738368
Block sizes: 4096
Numbers of processes: 32 48 64 72 84 96
Read/write type options: read write rw randread randwrite randrw
Async I/O engines: 
Async I/O depths: 
Sync I/O engines: sync mmap



In [39]:
print(open("../../data/2025-03-11-time-21-44-10-third-run-limmem/fio-config.txt").read())

Total file size during runs: 34359738368
Block sizes: 4096
Numbers of processes: 64
Read/write type options: read write rw randread randwrite randrw
Sync I/O engines: mmap
Cgrouped memory limits: 256M 512M 1G 2G 4G 8G 16G 32G 64G
Options for DIRECT: 0 1



# Process scaling

In [41]:
sq("""
    SELECT cnproc, cdevice, readBW, writeBW, USR_UTIL_perc, SYS_UTIL_perc, IOW_UTIL_perc, IDL_UTIL_perc, CPU_UTIL_perc 
    FROM master3Processes
    WHERE crw='read' AND cioengine='mmap'
    ORDER BY cdevice, cnproc
""")

,cnproc,cdevice,readBW,writeBW,USR_UTIL_perc,SYS_UTIL_perc,IOW_UTIL_perc,IDL_UTIL_perc,CPU_UTIL_perc
0,32,ssd,1.5 GB,0.0 B,0.581500,1.37350,25.666500,72.3620,27.63800
1,48,ssd,2.0 GB,0.0 B,0.604286,3.09036,18.885700,77.4157,22.58430
2,64,ssd,2.5 GB,0.0 B,1.578330,5.20750,40.492500,52.7042,47.29580
3,72,ssd,2.6 GB,0.0 B,0.842000,6.24733,27.816700,65.0913,34.90870
4,84,ssd,2.7 GB,0.0 B,0.938750,7.62125,23.334400,68.1069,31.89310
5,96,ssd,2.9 GB,0.0 B,1.257140,8.20929,23.476400,67.0550,32.94500
6,32,zram,9.0 GB,0.0 B,3.615000,22.92000,0.000000,73.4300,26.57000
7,48,zram,8.3 GB,0.0 B,0.850870,5.83783,0.003043,93.2283,6.77174
8,64,zram,9.5 GB,0.0 B,3.155000,37.43500,0.005000,59.3950,40.60500
9,72,zram,6.8 GB,0.0 B,1.688180,16.17270,0.009091,82.1291,17.87090


In [42]:
sq("""
    SELECT cnproc, cdevice, readBW, writeBW, USR_UTIL_perc, SYS_UTIL_perc, IOW_UTIL_perc, IDL_UTIL_perc, CPU_UTIL_perc 
    FROM master3Processes
    WHERE crw='read' AND cioengine='sync'
    ORDER BY cdevice, cnproc
""")

,cnproc,cdevice,readBW,writeBW,USR_UTIL_perc,SYS_UTIL_perc,IOW_UTIL_perc,IDL_UTIL_perc,CPU_UTIL_perc
0,32,ssd,1.5 GB,0.0 B,0.581500,1.37350,25.666500,72.3620,27.63800
1,48,ssd,2.0 GB,0.0 B,0.604286,3.09036,18.885700,77.4157,22.58430
2,64,ssd,2.5 GB,0.0 B,1.578330,5.20750,40.492500,52.7042,47.29580
3,72,ssd,2.6 GB,0.0 B,0.842000,6.24733,27.816700,65.0913,34.90870
4,84,ssd,2.7 GB,0.0 B,0.938750,7.62125,23.334400,68.1069,31.89310
5,96,ssd,2.9 GB,0.0 B,1.257140,8.20929,23.476400,67.0550,32.94500
6,32,zram,9.0 GB,0.0 B,3.615000,22.92000,0.000000,73.4300,26.57000
7,48,zram,8.3 GB,0.0 B,0.850870,5.83783,0.003043,93.2283,6.77174
8,64,zram,9.5 GB,0.0 B,3.155000,37.43500,0.005000,59.3950,40.60500
9,72,zram,6.8 GB,0.0 B,1.688180,16.17270,0.009091,82.1291,17.87090


# Limited memory